In [1]:
import pandas
from bs4 import BeautifulSoup
import random
import time
from splinter import Browser

In [2]:
database="monsters.csv"
monsters = pandas.read_csv(database)
monsters = monsters.set_index('url')
monsters

,name,meta,str,dex,con,int,wis,cha,img,attributes,tidbits,descriptions
url,,,,,,,,,,,,
https://www.dndbeyond.com/monsters/16895-giant-spider,Giant Spider,"Large Beast, Unaligned",14 (+2),16 (+3),12 (+1),2 (-4),11 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/3...,Armor Class:14 (natural armor);Hit Points:26 (...,"Skills: Stealth +7;Senses: Blindsight 10 ft., ...","<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16841-dire-wolf,Dire Wolf,"Large Beast, Unaligned",17 (+3),15 (+2),15 (+2),3 (-4),12 (+1),7 (-2),https://www.dndbeyond.com/avatars/thumbnails/1...,Armor Class:14 (natural armor);Hit Points:37 (...,"Skills: Perception +3, Stealth +4;Senses: Pass...","<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16816-brown-bear,Brown Bear,"Large Beast, Unaligned",19 (+4),10 (+0),16 (+3),2 (-4),13 (+1),7 (-2),//www.dndbeyond.com/attachments/2/648/beast.jpg,Armor Class:11 (natural armor);Hit Points:34 (...,Skills: Perception +3;Senses: Passive Percepti...,"<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16991-rat,Rat,"Tiny Beast, Unaligned",2 (-4),11 (+0),9 (-1),2 (-4),10 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/1...,Armor Class:10;Hit Points:1 (1d4 - 1);Speed:20...,"Senses: Darkvision 30 ft., Passive Perception...","<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16891-giant-rat,Giant Rat,"Small Beast, Unaligned",7 (-2),15 (+2),11 (+0),2 (-4),10 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/1...,Armor Class:12;Hit Points:7 (2d6);Speed:30 ft.,"Senses: Darkvision 60 ft., Passive Perception...","<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16920-hawk,Hawk,"Tiny Beast, Unaligned",5 (-3),16 (+3),8 (-1),2 (-4),14 (+2),6 (-2),https://www.dndbeyond.com/avatars/thumbnails/9...,Armor Class:13;Hit Points:1 (1d4 - 1);Speed:10...,Skills: Perception +4;Senses: Passive Percepti...,"<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/17049-warhorse,Warhorse,"Large Beast, Unaligned",18 (+4),12 (+1),13 (+1),2 (-4),12 (+1),7 (-2),//www.dndbeyond.com/attachments/2/648/beast.jpg,Armor Class:11;Hit Points:19 (3d10 + 3);Speed:...,Senses: Passive Perception 11;Languages: --;Ch...,"<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16997-riding-horse,Riding Horse,"Large Beast, Unaligned",16 (+3),10 (+0),12 (+1),2 (-4),11 (+0),7 (-2),https://www.dndbeyond.com/avatars/thumbnails/9...,Armor Class:10;Hit Points:13 (2d10 + 2);Speed:...,Senses: Passive Perception 10;Languages: --;Ch...,"<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16844-draft-horse,Draft Horse,"Large Beast, Unaligned",18 (+4),10 (+0),12 (+1),2 (-4),11 (+0),7 (-2),//www.dndbeyond.com/attachments/2/648/beast.jpg,Armor Class:10;Hit Points:19 (3d10 + 3);Speed:...,Senses: Passive Perception 10;Languages: --;Ch...,"<div class=""mon-stat-block__description-block-..."


In [3]:
monsters_to_add = [
    'https://www.dndbeyond.com/monsters/16895-giant-spider', 
    'https://www.dndbeyond.com/monsters/16841-dire-wolf',
    'https://www.dndbeyond.com/monsters/16816-brown-bear',
    'https://www.dndbeyond.com/monsters/16991-rat',
    'https://www.dndbeyond.com/monsters/16891-giant-rat',
    'https://www.dndbeyond.com/monsters/16920-hawk',
    'https://www.dndbeyond.com/monsters/17049-warhorse',
    'https://www.dndbeyond.com/monsters/16997-riding-horse',
    'https://www.dndbeyond.com/monsters/16844-draft-horse',
    'https://www.dndbeyond.com/monsters/16874-giant-badger',
    'https://www.dndbeyond.com/monsters/16889-giant-owl',
    'https://www.dndbeyond.com/monsters/16900-giant-wolf-spider',
    'https://www.dndbeyond.com/monsters/16944-lion',
    'https://www.dndbeyond.com/monsters/17036-tiger',
    'https://www.dndbeyond.com/monsters/16976-panther',
    'https://www.dndbeyond.com/monsters/16881-giant-eagle',
    'https://www.dndbeyond.com/monsters/16974-owl',
    'https://www.dndbeyond.com/monsters/16888-giant-octopus',
    'https://www.dndbeyond.com/monsters/16968-octopus',
    'https://www.dndbeyond.com/monsters/16928-hunter-shark',
    'https://www.dndbeyond.com/monsters/16994-reef-shark',
    'https://www.dndbeyond.com/monsters/16896-giant-toad'
    # giant squid, squid, shark, octopus, jellyfish (box)
]

In [4]:
def get_attribute(block):
    header = block.find("div", class_="ability-block__heading").text.strip()
    value = block.find("span", class_="ability-block__score").text.strip()
    mod = block.find("span", class_="ability-block__modifier")
    if mod != None:
        value += f" {mod.text.strip()}"
    return value


In [5]:
def get_monster(url):
    monster = {"url":url}
    found_monster=False
    main_block = None
    while not found_monster:
        browser_name = random.choice(['chrome', 'firefox'])
        browser = Browser(browser_name)
        time.sleep(random.random()*5)
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        main_block = soup.find("div", class_="mon-stat-block")
        found_monster = (main_block != None)
        time.sleep(random.random()*60+60)
        if not found_monster:
            time.sleep(random.random()*600+50)
        browser.quit()
    monster["name"] = main_block.find("div", class_="mon-stat-block__name").text.strip()
    monster["img"] = soup.find("img", class_="monster-image")["src"]
    monster["meta"] = main_block.find("div", class_="mon-stat-block__meta").text.strip()
    attributes_str = ""
    attributes = main_block.find("div", class_="mon-stat-block__attributes").find_all("div", class_="mon-stat-block__attribute")
    for attribute in attributes:
        label = attribute.find("span", class_="mon-stat-block__attribute-label").text.strip()
        value = attribute.find("span", class_="mon-stat-block__attribute-data-value").text.strip()
        dataextra = attribute.find("span", class_="mon-stat-block__attribute-data-extra")
        if not dataextra == None:
            value = f"{value} {dataextra.text.strip()}"
        attributes_str += f";{label}:{value}"
    monster["attributes"] = attributes_str[1:]
    tidbits_str = ""
    tidbits = main_block.find("div", class_="mon-stat-block__tidbits").find_all("div", class_="mon-stat-block__tidbit")
    for tidbit in tidbits:
        label = tidbit.find("span", class_="mon-stat-block__tidbit-label").text.strip()
        data = tidbit.find("span", class_="mon-stat-block__tidbit-data").text.strip()
        tidbits_str += f";{label}: {data}"
    monster["tidbits"] = tidbits_str[1:]
    stat_blocks = main_block.find("div", class_="mon-stat-block__stat-block")
    monster["str"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--str"))
    monster["dex"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--dex"))
    monster["con"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--con"))
    monster["int"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--int"))
    monster["wis"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--wis"))
    monster["cha"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--cha"))
    descriptions_str = ""
    descriptions = main_block.find_all("div", class_="mon-stat-block__description-block")
    for description in descriptions:
        #heading = description.find("div", class_="mon-stat-block__description-block-heading").text.strip()
        content = description.find("div", class_="mon-stat-block__description-block-content")
        descriptions_str += ";;;" + str(content)
        monster["descriptions"] = descriptions_str[3:]
    return(monster)

In [6]:
def save_database(monsters, database):
    monsters["url"] = monsters.index
    monsters = monsters[['url','name', 'meta', "str","dex","con","int","wis","cha","img","attributes","tidbits","descriptions"]]
    monsters.to_csv(database, index=False)

In [7]:
def add_monster(monsters, url):
    if url in monsters.index:
        print(f"found {monsters.loc[url,'name']}, skipping")
        return(monsters)
    else:
        monster = get_monster(url)
        monster_df = pandas.DataFrame([monster]).set_index("url")
        print(f"downloading {monster['name']}")
        return(pandas.concat([monsters, monster_df]))

In [8]:
for monster_url in monsters_to_add:
    monsters = add_monster(monsters, monster_url)
    save_database(monsters, database)
monsters

found Giant Spider, skipping
found Dire Wolf, skipping
found Brown Bear, skipping
found Rat, skipping
found Giant Rat, skipping
found Hawk, skipping
found Warhorse, skipping
found Riding Horse, skipping
found Draft Horse, skipping
found Giant Badger, skipping
found Giant Owl, skipping
found Giant Wolf Spider, skipping
found Lion, skipping
found Tiger, skipping
found Panther, skipping
found Giant Eagle, skipping
found Owl, skipping
found Giant Octopus, skipping
found Octopus, skipping
found Hunter Shark, skipping
found Reef Shark, skipping
downloading Giant Toad


,name,meta,str,dex,con,int,wis,cha,img,attributes,tidbits,descriptions,url
url,,,,,,,,,,,,,
https://www.dndbeyond.com/monsters/16895-giant-spider,Giant Spider,"Large Beast, Unaligned",14 (+2),16 (+3),12 (+1),2 (-4),11 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/3...,Armor Class:14 (natural armor);Hit Points:26 (...,"Skills: Stealth +7;Senses: Blindsight 10 ft., ...","<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16895-giant...
https://www.dndbeyond.com/monsters/16841-dire-wolf,Dire Wolf,"Large Beast, Unaligned",17 (+3),15 (+2),15 (+2),3 (-4),12 (+1),7 (-2),https://www.dndbeyond.com/avatars/thumbnails/1...,Armor Class:14 (natural armor);Hit Points:37 (...,"Skills: Perception +3, Stealth +4;Senses: Pass...","<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16841-dire-...
https://www.dndbeyond.com/monsters/16816-brown-bear,Brown Bear,"Large Beast, Unaligned",19 (+4),10 (+0),16 (+3),2 (-4),13 (+1),7 (-2),//www.dndbeyond.com/attachments/2/648/beast.jpg,Armor Class:11 (natural armor);Hit Points:34 (...,Skills: Perception +3;Senses: Passive Percepti...,"<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16816-brown...
https://www.dndbeyond.com/monsters/16991-rat,Rat,"Tiny Beast, Unaligned",2 (-4),11 (+0),9 (-1),2 (-4),10 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/1...,Armor Class:10;Hit Points:1 (1d4 - 1);Speed:20...,"Senses: Darkvision 30 ft., Passive Perception...","<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16991-rat
https://www.dndbeyond.com/monsters/16891-giant-rat,Giant Rat,"Small Beast, Unaligned",7 (-2),15 (+2),11 (+0),2 (-4),10 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/1...,Armor Class:12;Hit Points:7 (2d6);Speed:30 ft.,"Senses: Darkvision 60 ft., Passive Perception...","<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16891-giant...
https://www.dndbeyond.com/monsters/16920-hawk,Hawk,"Tiny Beast, Unaligned",5 (-3),16 (+3),8 (-1),2 (-4),14 (+2),6 (-2),https://www.dndbeyond.com/avatars/thumbnails/9...,Armor Class:13;Hit Points:1 (1d4 - 1);Speed:10...,Skills: Perception +4;Senses: Passive Percepti...,"<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16920-hawk
https://www.dndbeyond.com/monsters/17049-warhorse,Warhorse,"Large Beast, Unaligned",18 (+4),12 (+1),13 (+1),2 (-4),12 (+1),7 (-2),//www.dndbeyond.com/attachments/2/648/beast.jpg,Armor Class:11;Hit Points:19 (3d10 + 3);Speed:...,Senses: Passive Perception 11;Languages: --;Ch...,"<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/17049-warhorse
https://www.dndbeyond.com/monsters/16997-riding-horse,Riding Horse,"Large Beast, Unaligned",16 (+3),10 (+0),12 (+1),2 (-4),11 (+0),7 (-2),https://www.dndbeyond.com/avatars/thumbnails/9...,Armor Class:10;Hit Points:13 (2d10 + 2);Speed:...,Senses: Passive Perception 10;Languages: --;Ch...,"<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16997-ridin...
https://www.dndbeyond.com/monsters/16844-draft-horse,Draft Horse,"Large Beast, Unaligned",18 (+4),10 (+0),12 (+1),2 (-4),11 (+0),7 (-2),//www.dndbeyond.com/attachments/2/648/beast.jpg,Armor Class:10;Hit Points:19 (3d10 + 3);Speed:...,Senses: Passive Perception 10;Languages: --;Ch...,"<div class=""mon-stat-block__description-block-...",https://www.dndbeyond.com/monsters/16844-draft...


In [9]:
save_database(monsters, database)